In [31]:
import os
import numpy as np
import pydrake
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import MultibodyPlant
from pydrake.multibody.inverse_kinematics import InverseKinematics
from pydrake.math import RigidTransform, RotationMatrix
from pydrake.all import MathematicalProgram, SolverType, Solve, SolverOptions

## Construct MultibodyPlant

In [32]:
path_to_urdf = os.path.join(os.getcwd(), "iiwa14_simplified_collision.urdf")
ee_frame_name = "iiwa_link_ee"

# setup the MultibodyPlant
plant = MultibodyPlant()
iiwa_model = Parser(plant).AddModelFromFile(path_to_urdf, model_name='robot')

world_frame = plant.world_frame()
# weld base joint to world
plant.WeldFrames(A=world_frame, B=plant.GetFrameByName('base', iiwa_model))
plant.Finalize()

ee_frame = plant.GetFrameByName(ee_frame_name, iiwa_model)

# create default context
context = plant.CreateDefaultContext()

In [33]:
def compute_forward_kinematics(plant, q):
    context = plant.CreateDefaultContext()
    plant.SetPositions(context, q)
    T_W_EE = plant.CalcRelativeTransform(context, frame_A=world_frame, frame_B=ee_frame)
    return T_W_EE.matrix()

In [34]:
q = np.zeros(7)
q[1] = 0.5
T_W_E = compute_forward_kinematics(plant, q)
print(T_W_E)
target_pos = T_W_E[:3, 3]

[[ 4.79425539e-01  5.20961776e-16 -8.77582562e-01  4.53536560e-01]
 [-1.05581052e-16  1.00000000e+00  5.35953588e-16 -2.88245255e-16]
 [ 8.77582562e-01 -1.64293747e-16  4.79425539e-01  1.19019310e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


## Setup IK program

In [37]:
ik = InverseKinematics(plant)
q_vars = ik.q() # variables in optimization problem
mp = ik.prog() # mathematical program object

pos_tol = 1e-3

p_WF = T_W_E[:3, 3]
F = ee_frame
W = world_frame
# add position constraint
ik.AddPositionConstraint(
            frameB=F, p_BQ=np.zeros(3),
            frameA=W, p_AQ_lower=p_WF - pos_tol, p_AQ_upper=p_WF + pos_tol)

res = Solve(mp)
print("success:", res.is_success())
print(res.get_solver_id().name())
q_val = res.GetSolution(q_vars)
print("q_val", q_val)
print("solution result:", res.get_solution_result())

success: True
SNOPT/f2c
q_val [ 0.          0.49377164  0.         -0.00855533  0.          0.00204851
  0.        ]
solution result: SolutionResult.kSolutionFound
